# Salient-Object-Detection
This is tensorflow implementation for cvpr2017 paper "Deeply Supervised Salient Object Detection with Short Connections"

### [Pretrained Model](https://drive.google.com/open?id=0B6l9O8aWij8fUGtVNldUTXA4eHc)

In [ ]:
import os, sys
import tensorflow as tf
import numpy as np
import imageio
from skimage.transform import rescale, resize, downscale_local_mean

import warnings
warnings.filterwarnings('ignore')

from types import SimpleNamespace
from scipy import misc

In [ ]:
g_mean = np.array(([126.88,120.24,112.19])).reshape([1,1,3])
OUTPUT = "./salient/output"
INPUT = "./images/original/1C4PJLCX8KD22313103.jpeg"

In [ ]:
def rgba2rgb(img):
    return img[:,:,:3]*np.expand_dims(img[:,:,3],2)

In [ ]:
def make_alpha(args):
    with tf.Session(config=tf.ConfigProto()) as sess:
        saver = tf.train.import_meta_graph('./salient/graph/my-model.meta')
        saver.restore(sess,tf.train.latest_checkpoint('./salient/model'))
        image_batch = tf.get_collection('image_batch')[0]
        pred_mattes = tf.get_collection('mask')[0]

        if args.rgb_folder:
            rgb_pths = os.listdir(args.rgb_folder)
            for rgb_pth in rgb_pths:
                rgb = imageio.imread(os.path.join(args.rgb_folder,rgb_pth))
                
                if rgb.shape[2]==4:
                    rgb = rgba2rgb(rgb)
                origin_shape = rgb.shape
                rgb = np.expand_dims(misc.imresize(rgb.astype(np.uint8),[320,320,3],interp="nearest").astype(np.float32)-g_mean,0)

                feed_dict = {image_batch:rgb}
                pred_alpha = sess.run(pred_mattes,feed_dict = feed_dict)
                final_alpha = misc.imresize(np.squeeze(pred_alpha),origin_shape)
                imageio.imwrite(os.path.join(output_folder,rgb_pth),final_alpha)

        else:
            rgb = imageio.imread(args.rgb)
            if rgb.shape[2]==4:
                rgb = rgba2rgb(rgb)
            origin_shape = rgb.shape[:2]
            rgb = np.expand_dims(misc.imresize(rgb.astype(np.uint8),[320,320,3],interp="nearest").astype(np.float32)-g_mean,0)

            feed_dict = {image_batch:rgb}
            pred_alpha = sess.run(pred_mattes,feed_dict = feed_dict)
            final_alpha = misc.imresize(np.squeeze(pred_alpha),origin_shape)
            imageio.imwrite(os.path.join(OUTPUT,'alpha.png'),final_alpha)

In [ ]:
d = {'rgb': INPUT, 'rgb_folder': False}
n = SimpleNamespace(**d)
make_alpha(n)